### SocialAL Models
# Data simulation using best-fit parameters - multiple subjects
KLS 8.30.19; update 7.7.22; 10.31.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import norm
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run ../common_functions.ipynb

In [4]:
run ../double_alpha_decay_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 
        
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(t)
        for x in options:
            Probs[x] = Probs[x] + decay * (0.5 - Probs[x])
            EVs[x] =  update_value(Probs[x]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2, .3, 2, .4])

### Pull in best-fit parameters from participants

In [11]:
dt1 = pd.read_csv('../../../output/two_alpha_with_decay_model_params_younger.csv')
dt2 = pd.read_csv('../../../output/two_alpha_with_decay_model_params_older.csv')
dt = pd.concat([dt1, dt2], ignore_index = True)
print(dt)

          id  alpha_gain  alpha_loss       beta     decay       -LLH
0   sub-1009    0.001106    0.001341  20.000000  0.000000  57.040450
1   sub-1021    0.009335    0.025581  20.000000  0.053900  30.456762
2   sub-1035    1.000000    0.535426   0.387557  0.008331  41.896743
3   sub-1034    0.093360    0.190526   2.336965  0.068830  35.488460
4   sub-1020    0.004375    0.054778  20.000000  0.041766  29.820692
..       ...         ...         ...        ...       ...        ...
58  sub-2009    0.000000    0.002525  20.000000  0.134186  61.362892
59  sub-2037    0.112370    1.000000   0.702037  0.137956  46.209416
60  sub-2023    0.007256    0.017130  20.000000  0.003410  31.599882
61  sub-2022    1.000000    0.000000   0.075135  1.000000  60.717200
62  sub-2036    0.734792    0.000000   0.097911  0.000000  61.106887

[63 rows x 6 columns]


In [12]:
a_gains = dt['alpha_gain']
a_losses = dt['alpha_loss']
betas = dt['beta']
decays = dt['decay']
parent_sub = dt['id']
print(a_gains)

0     0.001106
1     0.009335
2     1.000000
3     0.093360
4     0.004375
        ...   
58    0.000000
59    0.112370
60    0.007256
61    1.000000
62    0.734792
Name: alpha_gain, Length: 63, dtype: float64


### Simulate data

In [17]:
data = pd.DataFrame(columns= ['Trial','Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta', 'Decay'])
for p in range(len(decays)):
    for q in range(1,11):
        dt = sim_data(15, [a_gains[p], a_losses[p], betas[p], decays[p]])
        dt['Subject'] = parent_sub[p] + '_' + str(q)
        dt['Alpha_gain'] = [a_gains[p]] * 45
        dt['Alpha_loss'] = [a_losses[p]] * 45
        dt['Beta'] = [betas[p]] * 45
        dt['Decay'] = [decays[p]] * 45
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])

data.to_csv(path_or_buf = '../../../output/simulation/part_params/sim_2alpha_decay_model_data.csv', index = False)


In [18]:
print(data)

   Trial Stim_Sequence Choice Trustee_Response      Subject  Alpha_gain  \
0      1             1      1                0   sub-1009_1    0.001106   
1      2             2      4                0   sub-1009_1    0.001106   
2      3             0      4                1   sub-1009_1    0.001106   
3      4             1      2                1   sub-1009_1    0.001106   
4      5             0      3                1   sub-1009_1    0.001106   
..   ...           ...    ...              ...          ...         ...   
40    41             1      1                1  sub-2036_10    0.734792   
41    42             0      3                1  sub-2036_10    0.734792   
42    43             2      4                0  sub-2036_10    0.734792   
43    44             1      4                1  sub-2036_10    0.734792   
44    45             2      3                1  sub-2036_10    0.734792   

    Alpha_loss       Beta  Decay  
0     0.001341  20.000000    0.0  
1     0.001341  20.000000    